# Stimuli

In [45]:
import Prelude as P

import Data.Array.Repa as Arr
import Data.Array.Repa.Repr.Vector (V,)
import Data.Array.Repa.Eval (fromList,)

import Formatting as Fmt

import Temporal.Music as Mus
import Temporal.Music.Scales as Sc

In [56]:
scaleNames :: Array V DIM1 String
scaleNames = fromList (Z :. 2) ["12TET", "12TFarey"]

scaleGenerators :: Array V DIM1 (Hz -> Scale)
scaleGenerators = fromList (Z :. 2) [eqt, farey]
    where farey = fromIntervals 2 [1/1, 16/15, 9/8, 6/5, 5/4, 4/3, 17/12, 3/2, 8/5, 5/3, 16/9, 15/8]

scaleBases :: Array V DIM1 Hz
scaleBases = fromList (Z :. 7) $ P.map (scaleAt (eqt 2)) [0, 1, 2, 3, 4, 5, 6]

scales :: Array D DIM2 Scale
scales = traverse2 scaleGenerators scaleBases fs fe
    where fs (Z :. ng) (Z :. nb) = Z :. ng :. nb
          fe g b (Z :. ig :. ib) = g (Z :. ig) $ b (Z :. ib)
          
contextNotes :: Array D DIM2 [Score (Note a)]
contextNotes = Arr.map f scales
    where f s = P.map (setScale s . nx) [0, 4, 7]
    
probeSteps :: Array V DIM1 Step
probeSteps = fromList (Z :. 12) [0 .. 11]
    
probeNotes :: Array D DIM3 (Score (Note a))
probeNotes = traverse2 scales probeSteps fs fe
    where fs (Z :. ng :. nb) (Z :. nst) = Z :. ng :. nb :. nst
          fe sc st (Z :. ig :. ib :. ist) = setScale (sc (Z :. ig :. ib)) $ nx (st (Z :. ist))
          
stimulusChords :: Array D DIM3 (Score (Note a))
stimulusChords = Arr.zipWith f contextNotes' probeNotes
    where f cns pn = setLevel 1 . str 10 $ har (pn:cns)
          contextNotes' = extend (Z :. Arr.All :. Arr.All :. (12::Int)) contextNotes
          
stimulusFileNames :: Array D DIM3 String
stimulusFileNames = traverse3 scaleNames scaleBases probeSteps fs fe
    where fs (Z :. ng) (Z :. nb) (Z :. nst) = Z :. ng :. nb :. nst
          fe n b st (Z :. ig :. ib :. ist) =
              formatToString (string % "-" % fixed 3 % "-" % int % ".wav") (n (Z :. ig)) (b (Z :. ib)) (st (Z :. ist))

In [9]:
render $ head stimulusChords

[Event {eventStart = 0.0, eventDur = 10.0, eventContent = Note {noteVolume = Volume {volumeDiap = Diap {diapRange = (1.0e-5,1.0), diapLim = 5}, volumeAccent = 0.0, volumeLevel = 1}, notePitch = Pitch {pitchScale = Scale {scaleBase = 2.0, scaleOctave = 2.0, scaleSteps = [1.0,1.0594630943592953,1.122462048309373,1.189207115002721,1.2599210498948732,1.3348398541700344,1.4142135623730951,1.4983070768766815,1.5874010519681994,1.681792830507429,1.7817974362806785,1.8877486253633868]}, pitchBend = 0.0, pitchOctave = 0, pitchStep = 0}, noteParam = Nothing}},Event {eventStart = 0.0, eventDur = 10.0, eventContent = Note {noteVolume = Volume {volumeDiap = Diap {diapRange = (1.0e-5,1.0), diapLim = 5}, volumeAccent = 0.0, volumeLevel = 1}, notePitch = Pitch {pitchScale = Scale {scaleBase = 2.0, scaleOctave = 2.0, scaleSteps = [1.0,1.0594630943592953,1.122462048309373,1.189207115002721,1.2599210498948732,1.3348398541700344,1.4142135623730951,1.4983070768766815,1.5874010519681994,1.681792830507429,1.

In [41]:
import Prelude as P

import Csound.Base as C
import Csound.Sam as S

import Control.Monad (mapM_,)

In [42]:
data Pitch = C | Cis | D | Dis | E | F | Fis | G | Gis | A | Ais | B deriving (Show, Eq, Enum)

data ChromaticScale = ChromaticScale deriving (Show, Eq)

data Context = MajorTriad Pitch deriving Eq
instance Show Context where
    show (MajorTriad p) = show p ++ "MajorTriad"

type Probe = Pitch

data Stimulus = Stimulus Context Probe deriving Eq
instance Show Stimulus where
    show (Stimulus c p) = show c ++ "-" ++ show p

In [45]:
toEnum (mod 12 12) :: Pitch

C

In [27]:
class Element a where
    pitches :: a -> [Pitch]

instance Element Pitch where
    pitches p = [p]
    
instance Element ChromaticScale where
    pitches ChromaticScale = [C, Des, D, Es, E, F, Fis, G, As, A, Bb, B]
    
instance Element Context where
    pitches CMajorTriad = [C, E, G]

In [28]:
class Golden12 a where
    ratios :: a -> [Either Rational Double]
    coeffs :: a -> [D]
    coeffs x = map coeff $ ratios x
        where coeff (Left  r) = fromRational r
              coeff (Right d) = double d
    period :: Num b => a -> b
    period e = fromIntegral . foldl lcm 1 . map denominator . lefts $ ratios e

instance Element a => Golden12 a where
    ratios e = map ((g12 !!) . fromEnum) $ pitches e
        where lowerG12 = [1%1, 16%15, 10%9, 6%5, 5%4, 4%3]
              tritonus = sqrt 2
              upperG12 = reverse $ map (2 /) lowerG12
              g12      = map Left lowerG12 ++ [Right tritonus] ++ map Left upperG12

In [29]:
class ToSamples a where
    toSamples :: [D] -> a -> Either Sam (Sam, Sam)

instance ToSamples Pitch where
    toSamples (d:_) p = Left . lim d . str (head $ coeffs p) . loop . fromSig1 1 $ ticks 1 1
    
instance ToSamples Context where
    toSamples (d:_) c = Left . mean . lefts . map (toSamples [d]) $ pitches c
    
instance ToSamples Stimulus where
    toSamples (dc:dp:_) (Stimulus c p) = Right (cxt, del dc prb)
        where Left cxt = toSamples [dc + dp] c
              Left prb = toSamples [dp] p

In [30]:
class ToSignal a where
    toSignal :: D -> [D] -> a -> SE Sig2
    
instance ToSignal Stimulus where
    toSignal t ds s = do let Right (cxt, prb) = toSamples ds s
                         (cxtSig, _) <- runSam t cxt
                         (_, prbSig) <- runSam t prb
                         let d = (60 / t) * sum ds
                         return $ setDur d (cxtSig, prbSig)

In [32]:
ratios ChromaticScale

[Left (1 % 1),Left (16 % 15),Left (10 % 9),Left (6 % 5),Left (5 % 4),Left (4 % 3),Right 1.4142135623730951,Left (3 % 2),Left (8 % 5),Left (5 % 3),Left (9 % 5),Left (15 % 8)]

In [33]:
ratios CMajorTriad

[Left (1 % 1),Left (5 % 4),Left (3 % 2)]

In [34]:
ratios CMinorTriad

[Left (1 % 1),Left (6 % 5),Left (3 % 2)]

In [35]:
contextChords = [CMajorTriad, CMinorTriad]
probePitches  = pitches ChromaticScale

Tempo (bpm):

In [36]:
tempo = 120

Context length (periods):

In [37]:
cxtLen = 2

Probe length (context periods):

In [38]:
prbLen = 2 * cxtLen

Stimuli:

In [39]:
stimuli = [(Stimulus c p, period c) | c <- contextChords, p <- probePitches]
mapM_ (\(s, p) -> writeSnd (show s ++ ".wav") (toSignal tempo [cxtLen * p, prbLen * p] s)) stimuli